<a href="https://colab.research.google.com/github/anurag2947/GoIndex-Downloader/blob/master/GoIndex_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
!apt update
!pip install selenium
!apt install chromium-chromedriver

In [0]:
import os
import shutil
import urllib.parse
import sys

from time import sleep

import requests
from tqdm import tqdm

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver

from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def getLinks(pageUrl, path):
    all_links_text = ""
    dirs = ""
    driver.get(pageUrl)
    try:
      WebDriverWait(driver, 30).until(EC.invisibility_of_element_located((By.CLASS_NAME, "mdui-progress")))
    except:
      print("Site took too long to load")
      sys.exit()
    html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")

    page_source = BeautifulSoup(html, 'lxml')
    tags_a_list = page_source.find('ul').find_all_next('a')


    for link in tags_a_list:

        currentLink = domain + str(link.get('href'))

        # checking whether currentLink is a link to folder or file

        # link to folder
        if currentLink[-1] == '/':
            sublink = currentLink[:currentLink.rindex('/')]
            dirname = sublink[sublink.rindex('/') + 1:]

            print(dirname)

            dirs = dirs + dirname + '\n'  # puts the name of currently made dir in dirs

            if os.path.exists(path + dirname):
                shutil.rmtree(path + dirname)
            os.makedirs(path + dirname)

            getLinks(currentLink, path + dirname + "/")

        # Else it is a link to file
        else:
            all_links_text = all_links_text + currentLink.replace("?a=view", "").replace(" ", "%20") + "\n"

    # writing all links to links.txt
    if all_links_text != "":
        all_links_text = all_links_text[:all_links_text.rindex('\n')]
        fileLinks = open(path + "links.txt", 'w')
        fileLinks.write(all_links_text)
        fileLinks.close()

    # writing all directories names to dirs.txt
    if dirs != "":
        dirs = dirs[:dirs.rindex('\n')]
        fileDirs = open(path + "dirs.txt", 'w')
        fileDirs.write(dirs)
        fileDirs.close()


def file_downloader(download_url, file_store_path, file_name):
    r = requests.get(download_url, stream=True)

    try:
        total_size = int(r.headers['content-length'])
    except:
        total_size = 1
    with open(file_store_path + file_name, 'wb') as f:
        for data in tqdm(iterable=r.iter_content(chunk_size=chunkSize), total=total_size / chunkSize, unit='KB'):
            f.write(data)


def dir_crawler(path):
    if os.path.isfile(path + "dirs.txt"):
        dirsFile = open(path + "dirs.txt", 'r')
        for dir in dirsFile:
            dir_crawler(path + dir.strip() + "/")

        dirsFile.close()

    if os.path.isfile(path + "links.txt"):
        linksFile = open(path + "links.txt", 'r')
        for link in linksFile:
            download_url = link.strip()
            file_name = urllib.parse.unquote(download_url.rsplit('/', 1)[1])
            if os.path.isfile(path+file_name):
                print("\""+file_name+"\""+" already downloaded")
                continue
            file_downloader(download_url, path, file_name)
        linksFile.close()



URL = input("Enter the URL: ")
if URL[-1]!="/":
  URL = URL + "/"
password = input("Enter password if any. Else press enter: ")
path = input("Enter the path to empty directory: ")

if path[-1]!='/':
  path = path + "/"

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

if password:
  try:
      print("\n\n::Checking Authentication::")
      driver.get(URL)
      WebDriverWait(driver,5).until(EC.alert_is_present(),'Timed out waiting for alerts to appear')
      obj = driver.switch_to.alert
      obj.send_keys("index")
      obj.accept()
      print("\n::Authentication Successful::")
  except:
      print("\n::Authentication Error::")


print("████████████████████████████████████████████\n\n\n::Fetching Links & Creating Folders::\n")


domain = URL.split(".dev")[0] + ".dev" #get domain name from URL
getLinks(URL, path)
driver.quit()
print("\n::Link Fetch and Folders Creation Complete::")

print("████████████████████████████████████████████\n\n\n::Downloading::\n")
chunkSize = 1024
try:
    dir_crawler(path)
    print("\n\n::Download Complete::\n████████████████████████████████████████████")
except:
    print("\n\n::Download Error::\n████████████████████████████████████████████")
